In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

import cv2

import os

import sys

import matplotlib.pyplot as plt

import numpy as np

In [ ]:
print("Python version:", sys.version)
print("TensorFlow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print(tf.config.list_physical_devices("GPU"))
print(device_lib.list_local_devices())

In [ ]:
img_array = cv2.imread("Dataset/train/angry/Training_3908.jpg")

In [ ]:
img_array.shape

In [ ]:
#print(img_array)

In [ ]:
plt.imshow(img_array)

In [ ]:
Datadirectory = "Dataset/train/"

In [ ]:
Classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

In [ ]:
for _ in Classes:
    path = os.path.join(Datadirectory, _)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        # backToRGB = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
        plt.show();
        break
    break

In [ ]:
img_size = 224
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(cv2.cvtColor(new_array,  cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
new_array.shape

### Real all the image and convert them to array
   

In [ ]:
training_Data = []

def create_training_Date():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass

In [ ]:
create_training_Date()

In [ ]:
print(len(training_Data))

In [ ]:
import random
random.shuffle(training_Data)

In [ ]:
X = [] # data/feature
y = [] # label

for features, label in training_Data:
    X.append(features);
    y.append(label);

X = np.array(X).reshape(-1, img_size, img_size, 3) # Converted to 4D

In [ ]:
X.shape

### Normalize the data

In [ ]:
X = X/255.0 # Normalizing it 

In [ ]:
y[0]

In [ ]:
Y = np.array(y)

In [ ]:
Y.shape

## Deep learning Model for training - Transfer learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.MobileNetV2() ## Pretrained Model

In [ ]:
model.summary()

### Transfer Learning - Tuning , weights will start from last check point

In [ ]:
base_input = model.layers[0].input
base_output = model.layers[-2].output

In [ ]:
base_output

In [ ]:
final_output = layers.Dense(128)(base_output) ## New layer Added
final_output = layers.Activation("relu")(final_output) ## activation layer
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation("relu")(final_output)
final_output = layers.Dense(7, activation = "softmax")(final_output) ## Classification layer

In [ ]:
final_output

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = base_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
with tf.device("/GPU:0"):
    new_model.fit(X, Y, epochs=25)